In [1]:
"""Python WPS execute"""
from owslib.wps import WebProcessingService, monitorExecution, printInputOutput
from os import system

In [2]:
#import owslib
#owslib.__version__ # version must be >=0.10.3

In [2]:
wps_url = "http://localhost:8093/wps"
#wps_url = "http://birdhouse-lsce.extra.cea.fr:8093/wps"
wps = WebProcessingService(url=wps_url, verbose=False)

In [4]:
print wps.identification.title

Flyingpigeon


In [5]:
for process in wps.processes:
    print '%s : \t %s' % (process.identifier, process.abstract)

visualisation : 	 Just testing a nice script to visualise some variables
extractpoints : 	 Extract Timeseries for specified coordinates from grid data
subset_countries : 	 This process returns only the given polygon from input netCDF files.
eobs_to_cordex : 	 downloads EOBS data in adaped CORDEX format
ensembleRobustness : 	 Calculates the robustness as the ratio of noise to signal in an ensemle of timeseries


In [14]:
p = wps.describeprocess(identifier='ensembleRobustness')
for input in p.dataInputs:
    printInputOutput(input)
    print '\n'

 identifier=resource, title=NetCDF Files, abstract=NetCDF Files, data type=ComplexData
 Supported Value: mimeType=application/x-netcdf, encoding=None, schema=None
 Default Value: mimeType=application/x-netcdf, encoding=None, schema=None 
 minOccurs=1, maxOccurs=100


 identifier=start, title=Start Year, abstract=Beginn of the analysed period (e.g 1971; if not set, the first consistend year of the ensemble will be taken), data type=//www.w3.org/TR/xmlschema-2/#integer
 Any value allowed
 Default Value: 1971 
 minOccurs=0, maxOccurs=1


 identifier=end, title=End Year, abstract=End of the analysed period (e.g. 2050 if not set, the last consistend year of the ensemble will be taken), data type=//www.w3.org/TR/xmlschema-2/#integer
 Any value allowed
 Default Value: 2005 
 minOccurs=0, maxOccurs=1


 identifier=timeslice, title=Time slice, abstract=Time slice (in years) for robustness reference (default=10)), data type=//www.w3.org/TR/xmlschema-2/#integer
 Any value allowed
 Default Value: 

In [3]:
path = 'file:///home/nils/data/EUR-44/'

files = ['tas_EUR-44_MPI-M-MPI-ESM-LR_historical_r1i1p1_CLMcom-CCLM4-8-17_v1_mon_200101-200512.nc',
'tas_EUR-44_MPI-M-MPI-ESM-LR_historical_r1i1p1_MPI-CSC-REMO2009_v1_mon_200101-200512.nc',
'tas_EUR-44_MPI-M-MPI-ESM-LR_historical_r2i1p1_MPI-CSC-REMO2009_v1_mon_200101-200512.nc'   
]

In [16]:
from os.path import join

execute = wps.execute(
    identifier="ensembleRobustness", #indices_clipping",
    inputs=[
       ("resource",path+files[0]),
       ("resource",path+files[1]),
       ("resource",path+files[2]),
        ("start",'2001'),
        ("end",'2005'),
       #("resource",path+files[3]),
       ("timeslice",'1'),
        ("variable",'tas')
           ])

monitorExecution(execute, sleepSecs=5)
print execute.getStatus()

for o in execute.processOutputs:
    print o.reference
    

ProcessSucceeded
http://localhost:8090/wpsoutputs/flyingpigeon/output_graphic-d243fb62-cc19-11e5-be95-ecf4bb9a61a2.png
http://localhost:8090/wpsoutputs/flyingpigeon/output_high-d243fb62-cc19-11e5-be95-ecf4bb9a61a2.nc
http://localhost:8090/wpsoutputs/flyingpigeon/output_low-d243fb62-cc19-11e5-be95-ecf4bb9a61a2.nc
http://localhost:8090/wpsoutputs/flyingpigeon/output_signal-d243fb62-cc19-11e5-be95-ecf4bb9a61a2.nc


In [25]:
sync_req_url = "{wps_url}?" +\
    "request=Execute" +\
    "&service=WPS" +\
    "&version=1.0.0" +\
    "&identifier=ensembleRobustness" +\
    "&DataInputs=resource={resource1};resource={resource2}" +\
    "&storeExecuteResponse=false" +\
    "&status=false"

In [ ]:
url=sync_req_url.format(
    wps_url=wps_url,
    resource1=files[0],
    resource2=files[1],)
print url   

In [7]:
import requests

In [8]:
r = ''
for i in range(1,4):
    r= '%sresource={resource%s};' % (r,i)
print r    

resource={resource1};resource={resource2};resource={resource3};


In [9]:
async_req_url = "{wps_url}?" +\
    "request=Execute" +\
    "&service=WPS" +\
    "&version=1.0.0" +\
    "&identifier=ensembleRobustness" +\
    "&DataInputs="+r+"start={start};end={end};timeslice={timeslice}"\
    "&storeExecuteResponse=true" +\
    "&status=true"

In [10]:
url=async_req_url.format(
    wps_url=wps_url,
    resource1=path+files[0],
    resource2=path+files[1],
    resource3=path+files[2],
    start='2001',
    end='2005',
    timeslice='1'
)
print url 

http://localhost:8093/wps?request=Execute&service=WPS&version=1.0.0&identifier=ensembleRobustness&DataInputs=resource=file:///home/nils/data/EUR-44/tas_EUR-44_MPI-M-MPI-ESM-LR_historical_r1i1p1_CLMcom-CCLM4-8-17_v1_mon_200101-200512.nc;resource=file:///home/nils/data/EUR-44/tas_EUR-44_MPI-M-MPI-ESM-LR_historical_r1i1p1_MPI-CSC-REMO2009_v1_mon_200101-200512.nc;resource=file:///home/nils/data/EUR-44/tas_EUR-44_MPI-M-MPI-ESM-LR_historical_r2i1p1_MPI-CSC-REMO2009_v1_mon_200101-200512.nc;start=2001;end=2005;timeslice=1&storeExecuteResponse=true&status=true


In [11]:
r = requests.get(url)
from lxml import etree
from io import BytesIO
tree = etree.parse(BytesIO(r.content))
#print etree.tostring(tree)
status_url = tree.getroot().get("statusLocation")
print status_url

http://localhost:8090/wpsoutputs/flyingpigeon/pywps-aa3dc508-cc19-11e5-be95-ecf4bb9a61a2.xml


In [13]:
r = requests.get(status_url)
print r.status_code
print r.text

200
<?xml version="1.0" encoding="utf-8"?>
<wps:ExecuteResponse xmlns:wps="http://www.opengis.net/wps/1.0.0" xmlns:ows="http://www.opengis.net/ows/1.1" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/wps/1.0.0 http://schemas.opengis.net/wps/1.0.0/wpsExecute_response.xsd" service="WPS" version="1.0.0" xml:lang="en-CA" serviceInstance="http://localhost:8093/wps?service=WPS&amp;request=GetCapabilities&amp;version=1.0.0" statusLocation="http://localhost:8090/wpsoutputs/flyingpigeon/pywps-aa3dc508-cc19-11e5-be95-ecf4bb9a61a2.xml">
    <wps:Process wps:processVersion="0.2">
        <ows:Identifier>ensembleRobustness</ows:Identifier>
        <ows:Title>Calculation of the robustness of an ensemle</ows:Title>
        <ows:Abstract>Calculates the robustness as the ratio of noise to signal in an ensemle of timeseries</ows:Abstract>
        <ows:Metadata xlink:title="LSCE" xlink:href="http://www.lsce.ipsl.f